# Modellierung von EV in Energiesystemen





------------------------------------------------
### Step 0: set up environments
------------------------------------------------

#### Create the environment from the file: ####

conda env create -f environment.yml

#### Activate the environment: ####
conda activate emobpy_py36


------------------------------------------------
### Step 1: Import
------------------------------------------------

In [1]:
##########
import os
import sys
from IPython.display import Markdown, display
import ipywidgets as widgets
import pandas as pd
##########
from emobpy import DataBase
from emobpy.init import copy_to_user_data_dir, create_project
##########
# Relative or absolute path to the folder containing the .py files
module_path = os.path.abspath(os.path.join('..'))
# Add to the system path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)
##########
# now you can import
from calculate_consumption import get_driving_profiles, load_bev_database, select_bev_models, run
from create_rules import create_emobpy_rules_yaml
from create_time_series import create_time_series_for_usergroups
import analyse_profiles as ap
##########

------------------------------------------------
### Step 2: General settings
------------------------------------------------

**Create initial project structure (only necessary once).**

In [2]:
copy_to_user_data_dir()  # Copy templates 
create_project('bev', 'base')  # Create project structure with 'base' template

Copy files from C:\Users\alsle\.conda\envs\emobpy_py36\lib\site-packages\emobpy\data\base
   C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks\bev\Instructions.rst
   C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks\bev\Step1Mobility.py
   C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks\bev\Step2DrivingConsumption.py
   C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks\bev\Step3GridAvailability.py
   C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks\bev\Step4GridDemand.py
   C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks\bev\Time-series_generation.ipynb
   C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks\bev\Visualize_and_Export.ipynb
   C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks\bev\config_files\DepartureDestinationTrip.csv
   C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks\bev\config_files\DistanceDurationTrip.csv
   C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks\bev\config_files\rules.yml
   C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks\bev\config_f

**Create a configuration file in a specific directory and define the necessary paths.**

In [3]:
#Don't change it. The structure is predefined.

base_path = os.getcwd()
display(Markdown(f"**base_path:** `{base_path}`"))
relative_path = os.path.join("bev", "config_files")
display(Markdown(f"**relative_path:** `{relative_path}`"))
# 🔗 Combine
config_path = os.path.join(base_path, relative_path)
db_folder = "db"

display(Markdown(f"**Config Files are under the following path:** `{config_path}`"))

**base_path:** `C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks`

**relative_path:** `bev\config_files`

**Config Files are under the following path:** `C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks\bev\config_files`

------------------------------------------------
### Step 3: Create Rules
------------------------------------------------

This code creates a rule definition file in YAML-compatible format (more precisely, as a JSON structure with the extension .yml) that can be used by the Emobpy library. This file contains mobility rules for different target groups (e.g., commuters, students). It describes:

- When and how often people travel.
- Where their first or last trip of the day goes.
- And how long they should spend at certain locations (e.g., “home,” “workplace”) at a minimum.

**A group of people, such as "commuters" or "students," can be added, deleted, or changed to suit your case.**

Add this section to the specifics_rules dictionary below to add a new group.


```json
specifics_rules = {
     "new_user1": {
            "weekday": {
                "overall_min_time_at": {"home": 9,"workplace":7.0},
                "overall_max_time_at":{"workplace":8.0},
            },
            "weekend": {
                "overall_min_time_at": {"home": 6},
            }
        }


In [4]:
# specific rules
# Example of group-specific additions:
# The user should adjust it to the case that should be studied.

specifics_rules = {
    "commuter": {
        "weekday": {
            "overall_min_time_at": {"home": 9,"workplace":7.0},
            "overall_max_time_at":{"workplace":8.0},
        },
        "weekend": {
            "overall_min_time_at": {"home": 6},
        }
    },
    "student": {
        "weekday": {
            "overall_min_time_at": {"home": 8, "workplace":6.0},
        },
        "weekend": {
            "overall_min_time_at": {"home": 10},
        }
    }
}

In [5]:
# Baseline rules, all groups have this, adjust it if you need
base_day_rules = {
    'n_trip_out': [1],
    'last_trip_to': {'home': True},
    'min_state_duration': {
        'home': 0.25, 'errands': 0.25, 'escort': 0.25,
        'shopping': 0.25, 'leisure': 0.25
    }
}

In [6]:
# Groups you want to study. If the group is not defined, you will receive an error.
groups = ["commuter", "student"]  # when added a new group => groups = ["commuter", "student", "new_user1"] 

In [7]:
# Call up function
create_emobpy_rules_yaml(config_path, groups, specifics_rules, base_day_rules)

Rules file saved at: C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks\bev\config_files\rules.yml ✅


In [8]:
#If you want to show the yaml rule file ++ Run this ++
import yaml

# Path to the YAML file
file_path = fr"{config_path}\\rules.yml"

# Load the YAML content
with open(file_path, "r", encoding="utf-8") as file:
    data = yaml.safe_load(file)

# Print the data
print(data)

{'commuter': {'weekday': {'n_trip_out': [1], 'last_trip_to': {'home': True}, 'min_state_duration': {'home': 0.25, 'errands': 0.25, 'escort': 0.25, 'shopping': 0.25, 'leisure': 0.25}, 'overall_min_time_at': {'home': 9, 'workplace': 7.0}, 'overall_max_time_at': {'workplace': 8.0}}, 'weekend': {'n_trip_out': [1], 'last_trip_to': {'home': True}, 'min_state_duration': {'home': 0.25, 'errands': 0.25, 'escort': 0.25, 'shopping': 0.25, 'leisure': 0.25}, 'overall_min_time_at': {'home': 6}}}, 'student': {'weekday': {'n_trip_out': [1], 'last_trip_to': {'home': True}, 'min_state_duration': {'home': 0.25, 'errands': 0.25, 'escort': 0.25, 'shopping': 0.25, 'leisure': 0.25}, 'overall_min_time_at': {'home': 8, 'workplace': 6.0}}, 'weekend': {'n_trip_out': [1], 'last_trip_to': {'home': True}, 'min_state_duration': {'home': 0.25, 'errands': 0.25, 'escort': 0.25, 'shopping': 0.25, 'leisure': 0.25}, 'overall_min_time_at': {'home': 10}}}}


------------------------------------------------
### Step 4:   Creates mobility time series for defined user groups
------------------------------------------------

In this step, time series can be created for defined user groups. To define the time series, provide the time series parameters: 
- **total_hours,** 
- **time_step_in_hrs,** 
- **and reference_date.**

In [9]:
total_hours = 168             # minimum 1 week
time_step_in_hrs = 0.25       # 15 minutes
reference_date = "01/01/2020" # format "dd/mm/yyyy"

In [10]:
# 🚀 Profile erzeugen
# Groups you want to study. If the group is not defined, you will receive an error.
user1 = "student"
m1 = create_time_series_for_usergroups(total_hours, time_step_in_hrs,reference_date,user1, config_path=config_path, db_folder=db_folder)
user2 = "commuter"
m2 = create_time_series_for_usergroups(total_hours, time_step_in_hrs,reference_date,user2, config_path=config_path, db_folder=db_folder)


🔄 Erstelle Mobilitätsprofil für: student
New profile running: BEV_student_W2_92c18
Progress: 100% [7 / 7] days
Profile done: BEV_student_W2_92c18
Elapsed time (min): 0.81
 
See Log files
C:\Users\alsle\AppData\Local\emobpy\emobpy\log\emobpy.log
✅ Profil gespeichert: student

🔄 Erstelle Mobilitätsprofil für: commuter
New profile running: BEV_commuter_W2_bcb96
Progress: 100% [7 / 7] days
Profile done: BEV_commuter_W2_bcb96
Elapsed time (min): 0.01
 
See Log files
C:\Users\alsle\AppData\Local\emobpy\emobpy\log\emobpy.log
✅ Profil gespeichert: commuter



In [11]:
# 🔍 Beispiel: Infos zu der Profil der ersten Gruppe anzeigen
m1.name

'BEV_student_W2_92c18'

In [12]:
m1.kind

'driving'

In [13]:
m1.description

'Mobilitätsprofil für student'

In [14]:
m1.timeseries

,hh,state,distance
date,,,
2020-01-01 00:00:00,0,home,0
2020-01-01 00:15:00,0.25,home,0
2020-01-01 00:30:00,0.5,home,0
2020-01-01 00:45:00,0.75,home,0
2020-01-01 01:00:00,1,home,0
...,...,...,...
2020-01-07 22:45:00,166.75,home,0
2020-01-07 23:00:00,167,home,0
2020-01-07 23:15:00,167.25,home,0


In [15]:
m1.profile.head()

,hr,state,departure,arrival,last_arrival,purpose,duration,weekday,category,distance,trip_duration
9.25,9.25,home,9.50,9.75,-3.00,errands,12.50,Wednesday,student,0.0,0.0
9.50,9.50,driving,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,12.0
12.50,12.50,errands,12.75,13.00,9.75,leisure,3.00,Wednesday,student,0.0,0.0
12.75,12.75,driving,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,10.0
13.00,13.00,leisure,13.25,13.50,13.00,errands,0.25,Wednesday,student,0.0,0.0


------------------------------------------------
### Step 5: Access Databank and plot profiles
------------------------------------------------

You create a DataBase object that searches the db folder.

This folder contains .pkl files with mobility time series

In [16]:
# get database path
db_path = os.path.join(base_path,db_folder)

# load profiles
db = ap.load_profiles(db_path)

# read profile names
profile_names = ap.list_profiles(db)

display(Markdown(f"**Databank folder is under the following path:** `{db_path}`"))

**Databank folder is under the following path:** `C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks\db`

In [17]:
#display profile names
display(Markdown(f"**Profile Names** `{profile_names}`"))

**Profile Names** `['BEV_commuter_W2_bcb96', 'BEV_student_W2_92c18']`

In [18]:
#Dropdown

display_names = profile_names

if profile_names[0] is not None:
    # 🔽 Dropdown definieren mit leerem Start
    dropdown = widgets.Dropdown(
        options=display_names,
        value= profile_names[0],  # default = leer
        description='🔽 Profil auswählen:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    output = widgets.Output()

    # ▶️ Anzeigen
    display(dropdown)
else:
    print("You need to create a profile alt least")


Dropdown(description='🔽 Profil auswählen:', layout=Layout(width='60%'), options=('BEV_commuter_W2_bcb96', 'BEV…

In [19]:
# Informationen zu einem Profil anzeigen (optional)
# info = ap.show_profile_info(db, dropdown.value)
# pprint(info)

**To show a different profile, select it from the dropdown bar.**

In [20]:
ap.run_profile_viewer(str(dropdown.value),db)

## 🔍 Profilansicht: `BEV_commuter_W2_bcb96`

Output()

------------------------------------------------
### Step 6: Set Vehicle model configuration and create consumption
------------------------------------------------

In this section, a specific electric vehicle model is selected from the emobpy database in order to calculate the energy consumption based on the driving profile. Both vehicle parameters (e.g. weight, cw value, battery size) and thermal properties (heating requirements, insulation, etc.) play a role here.

In [21]:
db = DataBase("db")
db.loadfiles("db")
profiles_list = list(db.db.keys())
BEVS = load_bev_database()[1] # Database class
BEV_Database = load_bev_database()[0] # Database that contains BEV models

In [22]:
profiles_list

['BEV_commuter_W2_bcb96', 'BEV_student_W2_92c18']

In [23]:
BEV_Database

,Manufacturer,Model,Year
0,Audi,e-tron 55 quattro,2019
1,Audi,e-tron 55 quattro,2020
2,Audi,e-tron 55 quattro Premium Plus,2019
3,Audi,e-tron 55 quattro Prestige,2019
4,Audi,e-tron Sportback 50 quattro,2020
...,...,...,...
174,Volkswagen,e-Golf SE,2020
175,Volkswagen,e-Golf SEL Premium,2015
176,Volkswagen,e-Golf SEL Premium,2016
177,Volkswagen,e-Golf SEL Premium,2017


In [24]:
#models_tupeln = [tuple(x) for x in models_to_study[["Manufacturer", "Model", "Year"]].values]
#models_tupeln[0]


In [25]:
#VW_ID3 = BEVS.model(models_tupeln[0])

In [26]:
#profiles_list[0]

In [27]:
# Hier sollten die gewünsten Modelle eingegeben werden
index_models_to_study = [178]
selected_models = select_bev_models(index_models_to_study,BEV_Database)
c = get_driving_profiles()
a = run(selected_models,c)

Fallback value 0.9 added for missing battery_charging_eff parameter.
Fallback value 0.95 added for missing battery_discharging_eff parameter.
Fallback value 0.95 added for missing transmission_eff parameter.
Fallback value 0.3 added for missing auxiliary_power parameter.
Fallback value 3.5 added for missing cabin_volume parameter.
Fallback value 1 added for missing hvac_cop_heating parameter.
Fallback value 2 added for missing hvac_cop_cooling parameter.
New profile running: BEV_commuter_W2_bcb96_Volkswagen_ID.3_2020_917ae
temp Kelvin DE 2016 Timezone: Europe/Berlin
pressure Pascal DE 2016 Timezone: Europe/Berlin
dew_point Kelvin DE 2016 Timezone: Europe/Berlin
Progress: 100% [14 / 14] trips
 
See Log files
C:\Users\alsle\AppData\Local\emobpy\emobpy\log\emobpy.log
Fallback value 0.9 added for missing battery_charging_eff parameter.
Fallback value 0.95 added for missing battery_discharging_eff parameter.
Fallback value 0.95 added for missing transmission_eff parameter.
Fallback value 0.

In [28]:
dataframe = BEVS.search_by_parameter()

Parameter: power
   brand                     model  year  value unit
0  Tesla             Model S P100D  2016  568.0   kw
1  Tesla  Model S Performance (SR)  2020  568.0   kw
2  Tesla  Model S Performance (AC)  2019  568.0   kw
3  Tesla   Model S P90DL Ludicrous  2016  568.0   kw
4  Tesla   Model S P90DL Ludicrous  2015  568.0   kw
5  Tesla              Model S P90D  2016  568.0   kw
6  Tesla              Model S P90D  2015  568.0   kw
7  Tesla   Model S P85DL Ludicrous  2015  568.0   kw
8  Tesla  Model S P85D 193kW+375kW  2015  568.0   kw
9  Tesla             Model S P100D  2018  568.0   kw


In [29]:
#setup to show all rows
#######
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#######

# Load and show time serie
#Load database
DB = DataBase('db')
DB.update()
DBc = DataBase('db')

#Load only consumption 
DBc.loadfiles_batch(kind="consumption", add_variables=['Trips'])

#select the first profile
profile_name = list(DBc.db.keys())[0]
print(f"Profilename : {profile_name}")

#create a dataframe with the selected profile
Consumption_TS = DB.db[profile_name]['timeseries']

#show dataframe with the selected profile
display(Consumption_TS)

Profilename : BEV_commuter_W2_bcb96_Volkswagen_ID.3_2020_917ae


,hh,state,distance,consumption,instant consumption in W,average power in W
date,,,,,,
2020-01-01 00:00:00,0,home,0,0.000000,0,0
2020-01-01 00:15:00,0.25,home,0,0.000000,0,0
2020-01-01 00:30:00,0.5,home,0,0.000000,0,0
2020-01-01 00:45:00,0.75,home,0,0.000000,0,0
2020-01-01 01:00:00,1,home,0,0.000000,0,0
2020-01-01 01:15:00,1.25,home,0,0.000000,0,0
2020-01-01 01:30:00,1.5,home,0,0.000000,0,0
2020-01-01 01:45:00,1.75,home,0,0.000000,0,0
2020-01-01 02:00:00,2,home,0,0.000000,0,0


In [30]:
### Define where data shoulde be saved ###
### default: \V2G_HTW_SS25\results\bev_timeseries\daten.csv ###
base_path = os.getcwd()
display(Markdown(f"**base_path:** `{base_path}`"))
BEV_results_path = os.path.abspath(os.path.join(base_path, '..', '..','..','results','bev_timeseries'))
display(Markdown(f"**BEV_results_path:** `{BEV_results_path}`"))
save_path = os.path.join(BEV_results_path,'daten.csv')
display(Markdown(f"**save_path:** `{save_path}`"))

**base_path:** `C:\git\github\V2G_HTW_SS25\models\emobpy\notebooks`

**BEV_results_path:** `C:\git\github\V2G_HTW_SS25\results\bev_timeseries`

**save_path:** `C:\git\github\V2G_HTW_SS25\results\bev_timeseries\daten.csv`

In [32]:
Consumption_TS.to_csv(save_path,sep=',')
display(Markdown(f"**The BEV-Data was saved to this path:** `{save_path}`"))

**The BEV-Data was saved to this path:** `C:\git\github\V2G_HTW_SS25\results\bev_timeseries\daten.csv`